# Üretici Ağlar

Tekrarlayan Sinir Ağları (Recurrent Neural Networks - RNN'ler) ve Uzun Kısa Süreli Bellek Hücreleri (Long Short Term Memory Cells - LSTM'ler) ile Gated Recurrent Units (GRU'lar) gibi kapılı hücre varyantları, dil modelleme için bir mekanizma sağladı, yani kelime sıralamasını öğrenebilir ve bir dizideki bir sonraki kelime için tahminlerde bulunabilirler. Bu, RNN'leri **üretici görevler** için kullanmamıza olanak tanır; örneğin, sıradan metin üretimi, makine çevirisi ve hatta görüntü altyazılama gibi.

Önceki birimde tartıştığımız RNN mimarisinde, her RNN birimi bir sonraki gizli durumu çıktı olarak üretiyordu. Ancak, her tekrarlayan birime başka bir çıktı daha ekleyebiliriz; bu, bir **dizi** (orijinal dizinin uzunluğuna eşit) üretmemizi sağlar. Dahası, her adımda bir girdi kabul etmeyen ve sadece bir başlangıç durum vektörü alarak bir çıktı dizisi üreten RNN birimleri de kullanabiliriz.

Bu not defterinde, metin üretmemize yardımcı olan basit üretici modeller üzerine odaklanacağız. Basitlik adına, harf harf metin üreten bir **karakter düzeyinde ağ** oluşturalım. Eğitim sırasında, bir metin korpusu almamız ve bunu harf dizilerine bölmemiz gerekecek.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Karakter kelime dağarcığı oluşturma

Karakter seviyesinde bir üretici ağ oluşturmak için metni kelimeler yerine bireysel karakterlere ayırmamız gerekir. Bu, farklı bir ayrıştırıcı tanımlayarak yapılabilir:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Haydi, veri setimizden metni nasıl kodlayabileceğimize dair bir örneğe bakalım:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Bir üretici RNN eğitmek

RNN'yi metin üretmek üzere eğitme yöntemimiz şu şekilde olacak. Her adımda, `nchars` uzunluğunda bir karakter dizisi alacağız ve ağdan her bir giriş karakteri için bir sonraki çıkış karakterini üretmesini isteyeceğiz:

![RNN'nin 'HELLO' kelimesini üretme örneğini gösteren bir görsel.](../../../../../translated_images/tr/rnn-generate.56c54afb52f9781d.webp)

Gerçek senaryoya bağlı olarak, *dizinin sonu* `<eos>` gibi bazı özel karakterleri de dahil etmek isteyebiliriz. Ancak bizim durumumuzda, ağı sonsuz metin üretimi için eğitmek istiyoruz, bu nedenle her bir dizinin boyutunu `nchars` tokenine eşit olarak sabitleyeceğiz. Sonuç olarak, her bir eğitim örneği `nchars` giriş ve `nchars` çıkıştan (giriş dizisinin bir sembol sola kaydırılmış hali) oluşacak. Minibatch, bu tür birkaç diziden oluşacak.

Minibatch'leri oluşturma yöntemimiz, uzunluğu `l` olan her bir haber metnini alıp, ondan tüm olası giriş-çıkış kombinasyonlarını üretmek olacak (bu kombinasyonlardan `l-nchars` kadar olacaktır). Bu kombinasyonlar bir minibatch oluşturacak ve minibatch boyutları her bir eğitim adımında farklılık gösterecektir.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Şimdi jeneratör ağını tanımlayalım. Bu ağ, önceki bölümde tartıştığımız herhangi bir yinelemeli hücreye (basit, LSTM veya GRU) dayanabilir. Bizim örneğimizde LSTM kullanacağız.

Ağ karakterleri girdi olarak aldığı ve kelime dağarcığı boyutu oldukça küçük olduğu için, bir gömme katmanına ihtiyacımız yok; tekil kodlanmış (one-hot-encoded) girdi doğrudan LSTM hücresine gidebilir. Ancak, karakter numaralarını girdi olarak verdiğimiz için, bunları LSTM'ye geçirmeden önce tekil kodlamamız gerekiyor. Bu, `forward` geçişi sırasında `one_hot` fonksiyonunu çağırarak yapılır. Çıktı kodlayıcı, gizli durumu tekil kodlanmış bir çıktıya dönüştürecek bir doğrusal katman olacaktır.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Eğitim sırasında, üretilen metni örneklemek isteyebiliriz. Bunu yapmak için, başlangıç dizesi `start` ile başlayarak, uzunluğu `size` olan bir çıktı dizesi üretecek `generate` fonksiyonunu tanımlayacağız.

Bu işlem şu şekilde çalışır: İlk olarak, başlangıç dizesinin tamamını ağdan geçiririz ve çıktı durumu `s` ile bir sonraki tahmin edilen karakter `out` elde edilir. `out` bir one-hot kodlaması olduğundan, sözlükteki karakterin indeksini `argmax` ile alırız (`nc`) ve ardından `itos` kullanarak gerçek karakteri bulur ve bunu sonuç karakterleri listesi `chars`a ekleriz. Bu süreç, gerekli sayıda karakter üretmek için `size` kez tekrarlanır.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Haydi eğitime başlayalım! Eğitim döngüsü, önceki örneklerimizin neredeyse aynısıdır, ancak doğruluk yerine her 1000 epoch'ta bir örneklenmiş üretilen metni yazdırıyoruz.

Kaybı hesaplama şekline özel dikkat gösterilmelidir. Tek-sıcak-kodlanmış çıktı `out` ve beklenen metin `text_out` (karakter indekslerinin listesi) verildiğinde kaybı hesaplamamız gerekiyor. Neyse ki, `cross_entropy` fonksiyonu, ilk argüman olarak normalleştirilmemiş ağ çıktısını ve ikinci argüman olarak sınıf numarasını bekler, ki bu tam olarak elimizde olan şeydir. Ayrıca minibatch boyutu üzerinde otomatik ortalama alma işlemi gerçekleştirir.

Eğitimi çok uzun sürmemesi için `samples_to_train` örnekleriyle sınırlandırıyoruz. Daha uzun bir eğitim denemenizi ve birkaç epoch boyunca çalışmayı denemenizi öneririz (bu durumda bu kodun etrafında başka bir döngü oluşturmanız gerekecektir).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Bu örnek zaten oldukça iyi bir metin üretiyor, ancak birkaç şekilde daha da geliştirilebilir:

* **Daha iyi minibatch oluşturma**. Veriyi eğitime hazırlama şeklimiz, her bir örnekten bir minibatch oluşturmak şeklindeydi. Bu ideal bir yöntem değil, çünkü minibatch'ler farklı boyutlarda oluyor ve bazıları metin `nchars` değerinden küçük olduğu için oluşturulamıyor. Ayrıca, küçük minibatch'ler GPU'yu yeterince verimli kullanmıyor. Daha akıllıca bir yaklaşım, tüm örneklerden büyük bir metin parçası almak, ardından tüm giriş-çıkış çiftlerini oluşturmak, bunları karıştırmak ve eşit boyutta minibatch'ler oluşturmak olacaktır.

* **Çok katmanlı LSTM**. 2 veya 3 katmanlı LSTM hücrelerini denemek mantıklı olabilir. Daha önceki bölümde bahsettiğimiz gibi, LSTM'nin her bir katmanı metinden belirli desenleri çıkarır ve karakter düzeyinde bir jeneratör durumunda, daha alt seviyedeki LSTM'nin heceleri çıkarmaktan, daha üst seviyelerin ise kelimeleri ve kelime kombinasyonlarını çıkarmaktan sorumlu olmasını bekleyebiliriz. Bu, LSTM yapıcısına katman sayısı parametresi geçirerek basitçe uygulanabilir.

* **GRU birimleri** ile denemeler yapmak isteyebilirsiniz ve hangilerinin daha iyi performans gösterdiğini görebilirsiniz. Ayrıca **farklı gizli katman boyutları** ile de deneyebilirsiniz. Çok büyük bir gizli katman aşırı öğrenmeye (örneğin, ağın metni birebir öğrenmesine) yol açabilir, küçük bir boyut ise iyi sonuçlar üretmeyebilir.


## Yumuşak metin üretimi ve sıcaklık

`generate` fonksiyonunun önceki tanımında, oluşturulan metindeki bir sonraki karakter olarak her zaman en yüksek olasılığa sahip karakteri seçiyorduk. Bu durum, metnin sık sık aynı karakter dizileri arasında "dönmesine" neden oluyordu, örneğin şu örnekte olduğu gibi:
```
today of the second the company and a second the company ...
```

Ancak, bir sonraki karakter için olasılık dağılımına baktığımızda, en yüksek olasılıklar arasındaki farkın çok büyük olmayabileceğini görebiliriz. Örneğin, bir karakterin olasılığı 0.2, diğerinin ise 0.19 olabilir, vb. Örneğin, '*play*' dizisindeki bir sonraki karakteri ararken, bir sonraki karakterin boşluk veya **e** (kelime *player*'daki gibi) olması eşit derecede olasıdır.

Bu durum, her zaman daha yüksek olasılığa sahip karakteri seçmenin "adil" olmadığını gösteriyor, çünkü ikinci en yüksek olasılığa sahip karakteri seçmek de anlamlı bir metne yol açabilir. Daha mantıklı olan, ağ çıktısının verdiği olasılık dağılımından karakterleri **örneklemektir**.

Bu örnekleme, **multinomial dağılımı** olarak adlandırılan bir yöntemi uygulayan `multinomial` fonksiyonu kullanılarak yapılabilir. Bu **yumuşak** metin üretimini uygulayan bir fonksiyon aşağıda tanımlanmıştır:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Sıcaklık olarak adlandırılan bir parametre daha tanıttık, bu parametre en yüksek olasılığa ne kadar sıkı bağlı kalmamız gerektiğini belirtmek için kullanılır. Eğer sıcaklık 1.0 ise, adil bir multinomial örnekleme yaparız ve sıcaklık sonsuza yaklaştığında - tüm olasılıklar eşit hale gelir ve bir sonraki karakteri rastgele seçeriz. Aşağıdaki örnekte, sıcaklığı çok fazla artırdığımızda metnin anlamsız hale geldiğini ve sıcaklık 0'a yaklaştığında "döngüsel" şekilde zor üretilmiş bir metni andırdığını gözlemleyebiliriz.



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul etmiyoruz.
